In [212]:
import pyodbc
import pandas as pd
import numpy as np
conn = pyodbc.connect('Driver={IBM i Access ODBC Driver}; '
       'SYSTEM = 192.168.0.161;'
        'Hostname=M3; '
        'Port=21; '
        'Protocol=TCPIP; '
        'Database=M3; '
        'UID=pud; '
        'PWD = Pud142;'
        ,autocommit=True)

In [209]:
sql="SELECT * FROM MATAN.MCWCCO"
mcwcco=pd.read_sql(sql,conn)
mcwcco_cols_values=mcwcco.columns.values



C:\Users\matan.s\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [222]:

res={}
for i in range(len(mcwcco)):
    row=mcwcco.iloc[i]
    
    temp=""
    for j in range(len(row)):
        if type(row[j])==np.float64:
            temp+=str(np.int64(row[j])).strip()
        else:
            temp+=str(row[j]).strip()
    if temp not in res:
        res[temp]=row
print(len(res))

85070


In [199]:
new_data=pd.read_csv("new_data.csv")

res={}
for i in mcwcco_cols_values:
        
        res[i]=[]
print((new_data.columns.values))

['fact' 'cost' 'price_type' 'work_center' 'date_update' ' ct1 ' ' ct5 '
 ' cost_type6 ' 'cost_type7' 'department' 'Unnamed: 10' 'Unnamed: 11'
 'Unnamed: 12']


In [ ]:
# cnt=0
# new_rows=[]
# for i in range(len(new_data)):
#     row=new_data.iloc[i]
#     new_row={}
    
#     for j in mcwcco_cols_values:
#         if j=="KECONO":
#             new_row[j]=1
#         if j=="KEFACI":
#             new_row[j]=row.fact
#         if j=="KECCOM":
#             new_row[j]=row.cost
#         if j=="KEPCTP":
#             new_row[j]=row.price_type
#         if j=="KEOBV1":
#             new_row[j]=row.work_center
#         if j=="KEOBV2":
#             new_row[j]=""
#         if j=="KEOBV2":
#             new_row[j]=""   
#         if j=="KEOBV3":
#             new_row[j]=""
#         if j=="KEFRDT":
#             new_row[j]=20220501
#         if j=="KEFRDT":
#             if row.price_type==1:
#                 new_row[j]=row.values[5]
#             if row.price_type==5:
#                 new_row[j]=row.values[6]
#             if row.price_type==6:
#                 new_row[j]=row.row.values[7]
#             if row.price_type==7:
#                 new_row[j]=row.row.values[8]
#         if j=="KERGDT":
#             new_row[j]=20221114
#         if j=="KERGTM":
#             new_row[j]=130000
#         if j=="KELMDT":
#             new_row[j]=20221114
#         if j=="KECHNO":
#             new_row[j]=1
#         if j=="KECHID":
#             new_row[j]="ITADMINMATAN"

#     new_rows.append(new_row)
    

In [200]:

_list=[]
for i in range(len(new_data)):
    row=new_data.iloc[i]
    
    temp=[1,row[0],row.cost,row.price_type,row.work_center,"","",row.date_update]
    if row.price_type==1:
        temp.append(row[5] )
    if row.price_type==5:
        temp.append(row[6])
    temp.append(20221114)
    temp.append(130000)
    temp.append(20221114)
    temp.append(1)
    temp.append("it_admin_matan")
    _list.append((temp))



[1, 100, 'B01', 1, 'WC411', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC412', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC413', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC450', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC500', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC512', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC525', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC709', '', '', 20221207, ' 7.00 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC752', '', '', 20221207, ' 22.93 ', 20221114, 130000, 20221114, 1, 'it_admin_matan']
[1, 100, 'B01', 1, 'WC777', '', '', 20221207, '

In [201]:
from random import randint,choice
cursor=conn.cursor()
l=[]
for i in range(10):
    temp=(i,choice(["matan","raz","romi","may"]),randint(1,40),randint(2000,2022))
    l.append(temp)
for row in l:
    cursor.execute("insert into MATAN.test1(id,name,age,date) values(?,?,?,?)",row)


(1, 700, 'B09   ', 1, 'WCP25          ', '               ', '               ', 20151101, 31.440000534057617, 20151130, 183212, 20151130, 38, 'VIRNA     ')